In [1]:
import time
import os
from os.path import exists


from log_logger import log_init, log_d, log_p
from s3_func import s3_list_pseudo_subdirs, s3_obj_exists
from xr_mosaic_func import xr_build_mosaic_ds, xr_write_geotiff_from_ds
from cog_func import cog_create_from_tif

#bucket = 'ws-enduser'
#out_prefix_path = '/caldera/projects/usgs/water/impd/butzer/'

def _build_full_output_path(primary_name, out_prefix_path):
    just_tif = primary_name.split('/')[-1]
    output = os.path.join(out_prefix_path, just_tif)
    print(f'OUTPUT=={output}')

    return output


def _file_already_here(path_to_file):
	return exists(path_to_file)

def caldera_write_geotiff_from_ds(DS, out_file_path):

    print(out_file_path)

    local_geotif = os.path.basename(out_file_path)

    print("local Geotiff", local_geotif)
    DS.rio.to_raster(out_file_path, compress='DEFLATE')

    #DS.rio.to_raster(local_geotif)
    #caldera_cog_create_from_tif(local_geotif, out_file_path)
    #os.remove(local_geotif)


def _run_command(cmd, verbose=False):
    if verbose:
        print(cmd)
    result = os.system(cmd)
    if result != 0:
        raise Exception('command "%s" failed with code %d.' % (cmd, result))


def cog_create_from_tif(src_tif,dst_cog):
    command = f'rio cogeo create {src_tif} {dst_cog}'
    _run_command(command)


def caldera_cog_create_from_tif(local_geotif, caldera_cog):
    command = f'rio cogeo create {local_geotif} {caldera_cog}'
    print(command)
    #_run_command(command)




def _do_just_one_day(product, target_year, day, subfolders, out_prefix_path):
        #log_d(f'justoneday {target_year} {day} {subfolders[0]}')

        year = target_year
        p = product
        tifs = []
        day3d = f'{day:03d}'
        print(len(subfolders))
        for fold in subfolders:
            print(fold)
            tif = f'{fold}/{year}/{p}_{year}{day3d}.tif'
            print(tif)
            tifs.append(tif)

        primary_name = tifs[0]  # first tif in list
        print(primary_name)
        print(f'...{primary_name} is placed at: {out_prefix_path}')
        out_obj = _build_full_output_path(primary_name, out_prefix_path)

        print(f'Output Item = {out_obj}')
        file_exists = _file_already_here(out_obj)
        if not file_exists:
            bucket='Nope'
            DS = xr_build_mosaic_ds(bucket, p, tifs)
            print(DS)

            # print(DS)
            if DS is not None:
                caldera_write_geotiff_from_ds(DS, out_obj)
                print('Next one')
                print('-------------------------------------------')
            else:
                print (f'No DS was created for {primary_name}')
        else:
                print (f'{primary_name} ALREADY Mosaiced wu wei')


def get_subfolders(prefix_path):
 
    folders = []
    rootdir = prefix_path
    for file in os.listdir(rootdir):
        d = os.path.join(rootdir, file)
        if os.path.isdir(d):
            #d = d + '/1984/'
            print(d)
            if 'chip' in d:
                folders.append(d)
    folders.sort()
    return folders

class Conus_mosaic:

    def __init__(self, in_prefix_path, year, out_prefix_path, products):
        log_d("hello from Conus_mosaic class")
        self.in_prefix_path = in_prefix_path
        self.products = products
        self.year = year
        self.out_prefix_path = out_prefix_path
        msg = f'Mosaicking {products} in {in_prefix_path} and put it in {out_prefix_path} for {year}'
        log_d(msg)

    def run_mosaic(self):
        log_d('run_mosaic')
        for prod in self.products:
            log_d(f'Mosaic this product: {prod}')

        if not self.in_prefix_path.endswith('/'):
            in_prefix_with_slash = self.in_prefix_path + '/'
        else:
            in_prefix_with_slash = self.in_prefix_path
        log_d(in_prefix_with_slash)
        subfolders = get_subfolders(in_prefix_with_slash)
        print("---"*20)
        print('len',len(subfolders))

        for f in subfolders:
            print(f)


        target_year=self.year
        product = self.products[0]
        out_prefix_path = self.out_prefix_path
        for day in range(1,367):
            start = time.time()
            print('DAY',day)
            print(len(subfolders))
            _do_just_one_day(product, target_year, day, subfolders, out_prefix_path)
            end = time.time()
            print(f'$$$$ MOSAIC took {end - start} SECONDS!')


In [ ]:
from log_logger import log_init, log_d, log_p

log = log_init('ET_MOSAIC', 'DEBUG')


in_prefix_path = '/caldera/projects/usgs/water/impd/skagone'
year = '1984'
out_prefix_path = '/caldera/projects/usgs/water/impd/skagone/conus_mos_completed/1984'
products = ['etasw']

mos = Conus_mosaic(in_prefix_path, year, out_prefix_path, products)
mos.run_mosaic()

14:13:29 ET_MOSAIC DEBUG: hello from Conus_mosaic class
14:13:29 ET_MOSAIC DEBUG: Mosaicking ['etasw'] in /caldera/projects/usgs/water/impd/skagone and put it in /caldera/projects/usgs/water/impd/skagone/conus_mos_completed/1984 for 1984
14:13:29 ET_MOSAIC DEBUG: run_mosaic
14:13:29 ET_MOSAIC DEBUG: Mosaic this product: etasw
14:13:29 ET_MOSAIC DEBUG: /caldera/projects/usgs/water/impd/skagone/


/caldera/projects/usgs/water/impd/skagone/chip39.71N-89.9E
/caldera/projects/usgs/water/impd/skagone/chip34.7N-89.9E
/caldera/projects/usgs/water/impd/skagone/chip34.7N-78.09E
/caldera/projects/usgs/water/impd/skagone/chip44.72N-78.09E
/caldera/projects/usgs/water/impd/skagone/chip49.724479188712984N-78.09E
/caldera/projects/usgs/water/impd/skagone/chip29.69N-89.9E
/caldera/projects/usgs/water/impd/skagone/chip29.69N-113.52E
/caldera/projects/usgs/water/impd/skagone/chip39.71N-113.52E
/caldera/projects/usgs/water/impd/skagone/chip34.7N-125.33203125E
/caldera/projects/usgs/water/impd/skagone/chip29.69N-125.33203125E
/caldera/projects/usgs/water/impd/skagone/conus_mos_completed
/caldera/projects/usgs/water/impd/skagone/chip49.724479188712984N-89.9E
/caldera/projects/usgs/water/impd/skagone/chip34.7N-101.71E
/caldera/projects/usgs/water/impd/skagone/chip34.7N-113.52E
/caldera/projects/usgs/water/impd/skagone/chip29.69N-101.71E
/caldera/projects/usgs/water/impd/skagone/chip44.72N-101.71E
/

/caldera/projects/usgs/water/impd/skagone/chip29.69N-78.09E/1984/etasw_1984001.tif 0.0254514217376709
SUCCESS _xr_open_rasterio_retry /caldera/projects/usgs/water/impd/skagone/chip29.69N-89.9E/1984/etasw_1984001.tif
/caldera/projects/usgs/water/impd/skagone/chip29.69N-89.9E/1984/etasw_1984001.tif 0.030611276626586914
SUCCESS _xr_open_rasterio_retry /caldera/projects/usgs/water/impd/skagone/chip34.7N-101.71E/1984/etasw_1984001.tif
/caldera/projects/usgs/water/impd/skagone/chip34.7N-101.71E/1984/etasw_1984001.tif 0.035321950912475586
SUCCESS _xr_open_rasterio_retry /caldera/projects/usgs/water/impd/skagone/chip34.7N-113.52E/1984/etasw_1984001.tif
/caldera/projects/usgs/water/impd/skagone/chip34.7N-113.52E/1984/etasw_1984001.tif 0.04012274742126465
SUCCESS _xr_open_rasterio_retry /caldera/projects/usgs/water/impd/skagone/chip34.7N-125.33203125E/1984/etasw_1984001.tif
/caldera/projects/usgs/water/impd/skagone/chip34.7N-125.33203125E/1984/etasw_1984001.tif 0.04481053352355957
SUCCESS _xr_op

/caldera/projects/usgs/water/impd/skagone/chip29.69N-101.71E/1984/etasw_1984002.tif 0.16916346549987793
SUCCESS _xr_open_rasterio_retry /caldera/projects/usgs/water/impd/skagone/chip29.69N-113.52E/1984/etasw_1984002.tif
/caldera/projects/usgs/water/impd/skagone/chip29.69N-113.52E/1984/etasw_1984002.tif 0.2516746520996094
SUCCESS _xr_open_rasterio_retry /caldera/projects/usgs/water/impd/skagone/chip29.69N-125.33203125E/1984/etasw_1984002.tif
/caldera/projects/usgs/water/impd/skagone/chip29.69N-125.33203125E/1984/etasw_1984002.tif 0.3650329113006592
SUCCESS _xr_open_rasterio_retry /caldera/projects/usgs/water/impd/skagone/chip29.69N-78.09E/1984/etasw_1984002.tif
/caldera/projects/usgs/water/impd/skagone/chip29.69N-78.09E/1984/etasw_1984002.tif 0.44806694984436035
SUCCESS _xr_open_rasterio_retry /caldera/projects/usgs/water/impd/skagone/chip29.69N-89.9E/1984/etasw_1984002.tif
/caldera/projects/usgs/water/impd/skagone/chip29.69N-89.9E/1984/etasw_1984002.tif 0.5632045269012451
SUCCESS _xr_o

/caldera/projects/usgs/water/impd/skagone/chip29.69N-101.71E/1984/etasw_1984003.tif 0.14170026779174805
SUCCESS _xr_open_rasterio_retry /caldera/projects/usgs/water/impd/skagone/chip29.69N-113.52E/1984/etasw_1984003.tif
/caldera/projects/usgs/water/impd/skagone/chip29.69N-113.52E/1984/etasw_1984003.tif 0.23653554916381836
SUCCESS _xr_open_rasterio_retry /caldera/projects/usgs/water/impd/skagone/chip29.69N-125.33203125E/1984/etasw_1984003.tif
/caldera/projects/usgs/water/impd/skagone/chip29.69N-125.33203125E/1984/etasw_1984003.tif 0.34084177017211914
SUCCESS _xr_open_rasterio_retry /caldera/projects/usgs/water/impd/skagone/chip29.69N-78.09E/1984/etasw_1984003.tif
/caldera/projects/usgs/water/impd/skagone/chip29.69N-78.09E/1984/etasw_1984003.tif 0.45653367042541504
SUCCESS _xr_open_rasterio_retry /caldera/projects/usgs/water/impd/skagone/chip29.69N-89.9E/1984/etasw_1984003.tif
/caldera/projects/usgs/water/impd/skagone/chip29.69N-89.9E/1984/etasw_1984003.tif 0.5873935222625732
SUCCESS _xr

SUCCESS _xr_open_rasterio_retry /caldera/projects/usgs/water/impd/skagone/chip29.69N-101.71E/1984/etasw_1984004.tif
/caldera/projects/usgs/water/impd/skagone/chip29.69N-101.71E/1984/etasw_1984004.tif 0.08778810501098633
SUCCESS _xr_open_rasterio_retry /caldera/projects/usgs/water/impd/skagone/chip29.69N-113.52E/1984/etasw_1984004.tif
/caldera/projects/usgs/water/impd/skagone/chip29.69N-113.52E/1984/etasw_1984004.tif 0.17874383926391602
SUCCESS _xr_open_rasterio_retry /caldera/projects/usgs/water/impd/skagone/chip29.69N-125.33203125E/1984/etasw_1984004.tif
/caldera/projects/usgs/water/impd/skagone/chip29.69N-125.33203125E/1984/etasw_1984004.tif 0.26799631118774414
SUCCESS _xr_open_rasterio_retry /caldera/projects/usgs/water/impd/skagone/chip29.69N-78.09E/1984/etasw_1984004.tif
/caldera/projects/usgs/water/impd/skagone/chip29.69N-78.09E/1984/etasw_1984004.tif 0.3454856872558594
SUCCESS _xr_open_rasterio_retry /caldera/projects/usgs/water/impd/skagone/chip29.69N-89.9E/1984/etasw_1984004.t